# 🤖 Agent Development - Risk Analyst & Compliance Officer

Welcome to Phase 2 & 3 of the project! In this notebook, you'll develop and test both AI agents:

1. **Risk Analyst Agent** (Chain-of-Thought prompting)
2. **Compliance Officer Agent** (ReACT prompting)

## 🎯 Learning Objectives
- Implement Chain-of-Thought prompting for systematic reasoning
- Build ReACT prompting for structured action-taking
- Handle structured JSON outputs from LLMs
- Create robust error handling and validation
- Test agents with real financial data

## 🚀 Setup and Environment

In [1]:
# Import required libraries
import os
import sys
import json
import openai
from dotenv import load_dotenv
from datetime import datetime

# Add src directory to Python path for module imports
sys.path.append(os.path.abspath('../src'))

# Load environment variables
load_dotenv()

print("📚 Libraries loaded!")
print("🔐 Environment variables loaded")
print("📂 Source directory added to Python path:", os.path.abspath('../src'))

📚 Libraries loaded!
🔐 Environment variables loaded
📂 Source directory added to Python path: /workspace/cd14685-fin-serv-agentic-c1-classroom/project/starter/src


In [2]:

def create_vocareum_openai_client():
    """
    Create an OpenAI client configured for Vocareum routing.
    
    This function handles the specific configuration required for Vocareum
    OpenAI API keys used in Udacity programs.
    
    Returns:
        openai.OpenAI: Configured OpenAI client instance
        
    Raises:
        ValueError: If OPENAI_API_KEY environment variable is not set
        ImportError: If openai package is not installed
    """
    try:
        import openai
    except ImportError:
        raise ImportError("openai package is required. Install with: pip install openai")
    
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError(
            "OPENAI_API_KEY environment variable not found. "
            "Get your Vocareum OpenAI API key from 'Cloud Resources' in your Udacity workspace."
        )
    
    if not api_key.startswith('voc-'):
        print("⚠️ Warning: API key doesn't start with 'voc-'. "
              "Make sure you're using a Vocareum OpenAI API key from your Udacity workspace.")
    
    # Configure client for Vocareum routing
    client = openai.OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=api_key
    )
    
    print("✅ OpenAI client initialized with Vocareum routing")
    print(f"🔑 API key: {api_key[:8]}...{api_key[-4:]}")
    print("📍 Base URL: https://openai.vocareum.com/v1")
    
    return client

In [3]:
# OpenAI Setup for Vocareum
import openai

# Option 1: Use the helper function from src package (recommended)
# Uncomment this after implementing foundation_sar.py:
# from src import create_vocareum_openai_client
client = create_vocareum_openai_client()

# Option 2: Manual setup (for early development)
# openai_api_key = os.getenv("OPENAI_API_KEY")
# if not openai_api_key:
#     print("⚠️ WARNING: No OpenAI API key found!")
#     print("Please set OPENAI_API_KEY in your .env file")
#     print("Get your Vocareum OpenAI API key from 'Cloud Resources' in your workspace")
# else:
#     # Vocareum requires routing through their servers
#     client = openai.OpenAI(
#         base_url="https://openai.vocareum.com/v1",
#         api_key=openai_api_key
#     )
#     print("✅ OpenAI client initialized with Vocareum routing")
#     print(f"🔑 API key: {openai_api_key[:8]}...{openai_api_key[-4:]}")
#     print("📍 Base URL: https://openai.vocareum.com/v1")
#     print("\n💡 Tip: After implementing foundation_sar.py, you can use:")
#     print("   from src import create_vocareum_openai_client")
#     print("   client = create_vocareum_openai_client()")

✅ OpenAI client initialized with Vocareum routing
🔑 API key: voc-7463...1538
📍 Base URL: https://openai.vocareum.com/v1


## 📊 Phase 1 Review: Load Foundation Components

Before building agents, let's ensure your foundation components are working.

In [4]:
# TODO: Import your implemented foundation components
# Uncomment and modify these imports once you've implemented foundation_sar.py

from foundation_sar import (
    CustomerData,
    AccountData,
    TransactionData,
    CaseData,
    RiskAnalystOutput,
    ComplianceOfficerOutput,
    ExplainabilityLogger,
    DataLoader,
    load_csv_data
)

print("📋 TODO: Import foundation components after implementing foundation_sar.py")
print("✅ Once imported, you can create sample cases for agent testing")

📋 TODO: Import foundation components after implementing foundation_sar.py
✅ Once imported, you can create sample cases for agent testing


In [5]:
# TODO: Test foundation components
# Once you've implemented foundation_sar.py, use this cell to:
# 1. Load CSV data
# 2. Create a DataLoader instance
# 3. Generate a sample case for testing agents
#https://github.com/Surveshchauhan/TRACE-Transactional-Risk-Analysis-Compliance-Engine

print("📋 TODO: Test foundation components")
print("1. Load CSV data from ../data/")
print("2. Create ExplainabilityLogger instance")
print("3. Create DataLoader instance")
print("4. Generate sample case for agent testing")
import pandas as pd
from foundation_sar import (
    ExplainabilityLogger,
    DataLoader,
    CustomerData,
    AccountData,
    TransactionData,
    CaseData
)

# 📋 Step 1: Load CSV data
def load_csv_data(data_dir: str):
    customers_df = pd.read_csv(f"{data_dir}/customers.csv")
    accounts_df = pd.read_csv(f"{data_dir}/accounts.csv")
    transactions_df = pd.read_csv(f"{data_dir}/transactions.csv")
    transactions_df = transactions_df.where(pd.notnull(transactions_df), None)
    customers_df = customers_df.where(pd.notnull(customers_df), None)



    return customers_df, accounts_df, transactions_df

# 📋 Step 2: Create ExplainabilityLogger instance
logger = ExplainabilityLogger("../outputs/audit_logs/agent_development.jsonl")

# 📋 Step 3: Create DataLoader instance
loader = DataLoader(logger)

# 📋 Step 4: Generate sample case for agent testing
customers_df, accounts_df, transactions_df = load_csv_data("../data/")
customers_df["ssn_last_4"] = customers_df["ssn_last_4"].astype(str)
customer_dict = customers_df.iloc[1].to_dict()
account_dicts = accounts_df.to_dict(orient="records")
transaction_dicts = transactions_df.to_dict(orient="records")
#print(customer_dict)
# Generate case
case = loader.create_case_from_data(
    customer_data=customer_dict,
    account_data=account_dicts,
    transaction_data=transaction_dicts
)

# ✅ Print summary
print(f"✅ Case created: {case.case_id}")
print(f"Customer: {case.customer.name}")
print(f"Accounts: {len(case.accounts)}")
print(f"Transactions: {len(case.transactions)}")
# Example workflow:


📋 TODO: Test foundation components
1. Load CSV data from ../data/
2. Create ExplainabilityLogger instance
3. Create DataLoader instance
4. Generate sample case for agent testing
✅ Case created: 023012c7-4758-4c6d-b09c-78a86113896c
Customer: Renee Blair
Accounts: 1
Transactions: 11


## 🔍 Phase 2: Risk Analyst Agent Development

The Risk Analyst Agent uses **Chain-of-Thought prompting** to systematically analyze suspicious activity patterns.

### 📚 Understanding Chain-of-Thought Prompting

Chain-of-Thought (CoT) prompting guides AI models through step-by-step reasoning:

1. **Explicit Steps**: Break complex reasoning into clear phases
2. **Sequential Logic**: Each step builds on previous ones
3. **Domain Expertise**: Frame AI as subject matter expert
4. **Structured Output**: Guide toward specific response format

In [6]:
# TODO: Test Chain-of-Thought prompt design
# This cell helps you design and test your CoT prompt structure

def design_cot_prompt():
    """Design and test Chain-of-Thought prompt for risk analysis"""
    
    system_prompt = """You are a Senior Financial Crime Risk Analyst specializing in Anti-Money Laundering (AML) investigations. Your task is to analyze suspicious activity using a structured 5-step reasoning framework. Think critically, document your rationale, and produce a clear classification decision.

            Follow this 5-step analysis framework:

            1. Data Review – Summarize key customer, account, and transaction details.
            2. Pattern Recognition – Identify suspicious behaviors or typologies.
            3. Regulatory Mapping – Link behavior to AML rules or precedents.
            4. Risk Quantification – Assess severity based on volume, velocity, geography, and profile.
            5. Classification Decision – Choose one category:
                "Structuring": "Transactions designed to avoid reporting thresholds",
                "Sanctions": "Potential sanctions violations or prohibited parties",
                "Fraud": "Fraudulent transactions or identity-related crimes",
                "Money_Laundering": "Complex schemes to obscure illicit fund sources", 
                "Other": "Suspicious patterns not fitting standard categories"

            Output your reasoning in this JSON format:

            {{
  "case_id": "<string>",
  "classification": "<Structuring | Sanctions | Fraud | Money_Laundering | Other>",
  "confidence_score": "<float between 0 and 1>",
  "reasoning": "<summary of rationale>",
  "key_indicators": ["<indicator_1>", "<indicator_2>", "..."],
  "risk_level": "<Low | Medium | High>",
  "analysis": {
    "data_review": "...",
    "pattern_recognition": "...",
    "regulatory_mapping": "...",
    "risk_quantification": "...",
    "classification_decision": "..."
  }
}

        """
    return system_prompt


# Test your prompt design
cot_prompt = design_cot_prompt()
print("🧠 Chain-of-Thought Prompt Design:")
print(cot_prompt[:200] + "...")


🧠 Chain-of-Thought Prompt Design:
You are a Senior Financial Crime Risk Analyst specializing in Anti-Money Laundering (AML) investigations. Your task is to analyze suspicious activity using a structured 5-step reasoning framework. Thi...


In [7]:
case

CaseData(case_id='023012c7-4758-4c6d-b09c-78a86113896c', customer=CustomerData(customer_id='CUST_0002', name='Renee Blair', date_of_birth='1978-05-14', ssn_last_4='7201', address='959 Janet Cape Apt. 413, South Joshuastad, GA 49021', customer_since='2023-03-03', risk_rating='Low', phone='(534)719-2832x764', occupation='Banker', annual_income=72005), accounts=[AccountData(account_id='CUST_0002_ACC_1', customer_id='CUST_0002', account_type='Money_Market', opening_date='2023-12-03', current_balance=92121.66, average_monthly_balance=172242.96, status='Active')], transactions=[TransactionData(transaction_id='TXN_B24455F3', account_id='CUST_0002_ACC_1', transaction_date='2025-02-18', transaction_type='Online_Transfer', amount=1615.06, description='Credit card payment', method='ATM', counterparty=None, location='Branch_Westside_Plaza'), TransactionData(transaction_id='TXN_0A18BAD4', account_id='CUST_0002_ACC_1', transaction_date='2025-02-22', transaction_type='Check_Deposit', amount=2564.42, 

In [8]:
# TODO: Implement and test Risk Analyst Agent - SIMPLE SMOKE TEST
# Students: Write a basic smoke test to verify your agent works

from risk_analyst_agent import RiskAnalystAgent
from foundation_sar import ExplainabilityLogger
def simple_risk_analyst_smoke_test():
    """
    STUDENT TODO: Write a simple smoke test for your Risk Analyst Agent
    
    This should be a basic test that:
    1. Creates a RiskAnalystAgent instance
    2. Creates simple test data (or uses mock data)
    3. Calls analyze_case() method
    4. Verifies the result has the expected structure
    5. Prints success/failure
    
    Keep this simple - just verify your agent doesn't crash and returns something reasonable!
    """
    print("🔍 Risk Analyst Smoke Test")
    print("📋 TODO: Import your RiskAnalystAgent")
    print("📋 TODO: Create agent instance: agent = RiskAnalystAgent(client, logger)")
    print("📋 TODO: Create simple test case data")  
    print("📋 TODO: Call: result = agent.analyze_case(test_case)")
    print("📋 TODO: Verify: result has classification, confidence_score, reasoning")
    print("📋 TODO: Print: 'SUCCESS' or 'FAILED' with details")
    
    # Example structure (uncomment and modify when ready):
    try:


# Step 2: Create ExplainabilityLogger instance
        explainability_logger = ExplainabilityLogger("../outputs/audit_logs/agent_development.jsonl")
        agent = RiskAnalystAgent(client, explainability_logger)
        # Create minimal test case...
        test_case = loader.create_case_from_data(
            customer_data=customer_dict,
            account_data=account_dicts,
            transaction_data=transaction_dicts
        )
        result = agent.analyze_case(test_case)
        print(f"✅ SUCCESS: Got classification '{result.classification}' with confidence {result.confidence_score}")
        print(result)
    except Exception as e:
        print(f"❌ FAILED: {e}")

simple_risk_analyst_smoke_test()

🔍 Risk Analyst Smoke Test
📋 TODO: Import your RiskAnalystAgent
📋 TODO: Create agent instance: agent = RiskAnalystAgent(client, logger)
📋 TODO: Create simple test case data
📋 TODO: Call: result = agent.analyze_case(test_case)
📋 TODO: Verify: result has classification, confidence_score, reasoning
📋 TODO: Print: 'SUCCESS' or 'FAILED' with details
✅ SUCCESS: Got classification 'Structuring' with confidence 0.65
classification='Structuring' confidence_score=0.65 reasoning='Customer has a low-risk rating but has made multiple transactions just below the $5000 reporting threshold. This could be an attempt to avoid detection.' key_indicators=['Multiple transactions below $5000', 'Low-risk customer'] risk_level='Medium'


### 🧪 Risk Analyst Testing Framework

In [9]:
# COMPREHENSIVE Risk Analyst Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import sys
import os

# Add tests directory to Python path for importing test modules
project_root = os.path.abspath('..')
tests_path = os.path.join(project_root, 'tests')


if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Added tests directory to Python path: {tests_path}")

def run_comprehensive_risk_analyst_tests():
    """
    Use pre-built comprehensive test suite to validate your Risk Analyst Agent
    
    These tests validate:
    - Agent initialization and configuration
    - Case analysis with valid inputs
    - JSON parsing and error handling
    - System prompt structure and content
    - API call parameters and responses
    - Helper method functionality
    """
    print("🧪 Comprehensive Risk Analyst Testing")
    #print("📋 TODO: Uncomment and run after implementing your Risk Analyst Agent")
    
    try:
        from test_risk_analyst import TestRiskAnalystAgent
        import pytest
        
        print("🔍 Loading comprehensive test suite...")
        
        # Run the test suite
        print("� Running Risk Analyst test suite...")
        #print(f"{tests_path}/test_risk_analyst.py")
        result = pytest.main([
            f"{tests_path}/test_risk_analyst.py", 
            "-v", 
            "--tb=short"
        ])
        
        if result == 0:
            print("✅ All Risk Analyst tests passed!")
        else:
            print("❌ Some tests failed. Check the output above for details.")
            
    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure you've implemented RiskAnalystAgent in src/risk_analyst_agent.py")

    # Quick preview of available tests
    try:
        from test_risk_analyst import TestRiskAnalystAgent
        import inspect
        
        # Get all test methods
        test_methods = [method for method in dir(TestRiskAnalystAgent) 
                    if method.startswith('test_')]
        
        print("📊 Preview of Comprehensive Risk Analyst Tests:")
        for method_name in test_methods[:5]:  # Show first 5
            method = getattr(TestRiskAnalystAgent, method_name)
            doc = method.__doc__ or method_name.replace('_', ' ').title()
            print(f"   • {doc}")
        if len(test_methods) > 5:
            print(f"   ... and {len(test_methods) - 5} more tests")
        # print("\n💡 These tests validate edge cases you might not think of!")
        # print("💡 Much more thorough than manual testing!")
    except Exception as e:
        print(f"ℹ️ Test suite will be available after implementing foundation_sar.py: {e}")
        # Call the function



📁 Added tests directory to Python path: /workspace/cd14685-fin-serv-agentic-c1-classroom/project/starter/tests


In [10]:
run_comprehensive_risk_analyst_tests()

🧪 Comprehensive Risk Analyst Testing
🔍 Loading comprehensive test suite...
� Running Risk Analyst test suite...
============================= test session starts ==============================
platform linux -- Python 3.13.0, pytest-8.4.1, pluggy-1.6.0 -- /opt/venv/bin/python
cachedir: .pytest_cache
rootdir: /workspace/cd14685-fin-serv-agentic-c1-classroom/project/starter
plugins: anyio-4.10.0
collecting ... collected 10 items

../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_agent_initialization PASSED [ 10%]
../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_analyze_case_success PASSED [ 20%]
../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_analyze_case_json_error PASSED [ 30%]
../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_extract_json_from_code_block PASSED [ 40%]
../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_extract_json_from_plain_text PASSED [ 50%]
../tests/test_risk_analyst.py::TestRiskAnalystAgent::test_extract_json_empty_respons

## ✅ Phase 3: Compliance Officer Agent Development

The Compliance Officer Agent uses **ReACT prompting** to generate regulatory-compliant SAR narratives.

### 📚 Understanding ReACT Prompting

ReACT (Reasoning + Action) prompting separates thinking and doing:

1. **Reasoning Phase**: Analyze situation and plan approach
2. **Action Phase**: Execute specific task with informed decisions
3. **Structured Workflow**: Consistent approach to complex tasks
4. **Regulatory Compliance**: Emphasis on meeting specific requirements

In [11]:
# TODO: Test ReACT prompt design
# This cell helps you design and test your ReACT prompt structure

def design_react_prompt():
    """Design and test ReACT prompt for compliance narratives"""
    
    system_prompt = """
You are a Senior Compliance Officer specializing in Anti-Money Laundering (AML) and suspicious activity reporting. Your task is to generate a concise, regulatory-compliant SAR narrative using the ReACT (Reasoning + Action) framework.

Follow this structured approach:

REASONING Phase:
1. Review the risk analyst's findings
2. Assess regulatory narrative requirements
3. Identify key compliance elements
4. Plan narrative structure

ACTION Phase:
1. Draft a concise SAR narrative (≤120 words)
2. Include specific customer details, transaction amounts, and dates
3. Reference the suspicious activity pattern
4. Use appropriate regulatory language and cite relevant rules

Your output must follow this JSON format:

{
  "case_id": "<string>",
  "narrative": "<SAR narrative text, ≤120 words>",
  "word_count": <integer>,
  "regulatory_citations": ["<citation_1>", "<citation_2>", "..."]
}

Ensure the narrative includes:
- Customer identification
- Description of suspicious activity
- Transaction details
- Justification for suspicion
- Regulatory terminology and citations (e.g., 31 CFR 1020.320, FinCEN SAR Instructions)

Be precise, compliant, and audit-ready.
"""
    return system_prompt


# Test your prompt design
react_prompt = design_react_prompt()
print("⚡ ReACT Prompt Design:")
print(react_prompt[:200] + "...")


⚡ ReACT Prompt Design:

You are a Senior Compliance Officer specializing in Anti-Money Laundering (AML) and suspicious activity reporting. Your task is to generate a concise, regulatory-compliant SAR narrative using the ReA...


In [12]:
# TODO: Implement and test Compliance Officer Agent - SIMPLE SMOKE TEST
# Students: Write a basic smoke test to verify your agent works

from compliance_officer_agent import ComplianceOfficerAgent
from foundation_sar import ExplainabilityLogger
def simple_compliance_officer_smoke_test():
    """
    STUDENT TODO: Write a simple smoke test for your Compliance Officer Agent
    
    This should be a basic test that:
    1. Creates a ComplianceOfficerAgent instance
    2. Creates simple test case and risk analysis data
    3. Calls generate_compliance_narrative() method  
    4. Verifies the result has a narrative with reasonable length
    5. Prints success/failure
    
    Keep this simple - just verify your agent doesn't crash and generates text!
    """
    print("✅ Compliance Officer Smoke Test")
    print("📋 TODO: Import your ComplianceOfficerAgent")
    print("📋 TODO: Create agent instance: agent = ComplianceOfficerAgent(client, logger)")
    print("📋 TODO: Create simple test case and sample risk analysis")
    print("📋 TODO: Call: result = agent.generate_compliance_narrative(case, risk_analysis)")
    print("📋 TODO: Verify: result has narrative, word count ≤ 120")
    print("📋 TODO: Print: 'SUCCESS' or 'FAILED' with details")
    
    # Example structure (uncomment and modify when ready):
    try:
        explainability_logger = ExplainabilityLogger("../outputs/audit_logs/agent_development.jsonl")
        agent = ComplianceOfficerAgent(client, explainability_logger)
        # Create test case and risk analysis...
        test_case = loader.create_case_from_data(
            customer_data=customer_dict,
            account_data=account_dicts,
            transaction_data=transaction_dicts
        )
        test_risk_analysis = RiskAnalystAgent(client, explainability_logger).analyze_case(test_case)
        result = agent.generate_compliance_narrative(test_case, test_risk_analysis)
        word_count = len(result.narrative.split())
        print(f"✅ SUCCESS: Generated {word_count} word narrative")
        print(f"Preview: {result.narrative[:100]}...")
    except Exception as e:
        print(f"❌ FAILED: {e}")

simple_compliance_officer_smoke_test()

✅ Compliance Officer Smoke Test
📋 TODO: Import your ComplianceOfficerAgent
📋 TODO: Create agent instance: agent = ComplianceOfficerAgent(client, logger)
📋 TODO: Create simple test case and sample risk analysis
📋 TODO: Call: result = agent.generate_compliance_narrative(case, risk_analysis)
📋 TODO: Verify: result has narrative, word count ≤ 120
📋 TODO: Print: 'SUCCESS' or 'FAILED' with details
❌ FAILED: Failed to generate compliance narrative: RiskAnalystAgent._format_accounts() missing 1 required positional argument: 'accounts'


### 🧪 Compliance Officer Testing Framework

In [ ]:
# COMPREHENSIVE Compliance Officer Testing - Import Pre-Built Test Suite
# Students: Use our comprehensive test suite instead of writing your own

import sys
import os

# Add tests directory to Python path for importing test modules (if not already added)
project_root = os.path.abspath('..')
tests_path = os.path.join(project_root, 'tests')
if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Tests directory in Python path: {tests_path}")

def run_comprehensive_compliance_officer_tests():
    """
    Use pre-built comprehensive test suite to validate your Compliance Officer Agent
    
    These tests validate:
    - Agent initialization and configuration
    - Narrative generation with valid inputs
    - Word count limits (≤120 words)
    - Regulatory citations inclusion
    - JSON parsing and error handling
    - ReACT prompt structure validation
    """
    print("🧪 Comprehensive Compliance Officer Testing")
    print("📋 TODO: Uncomment and run after implementing your Compliance Officer Agent")
    
    # Uncomment when your agent is ready:
    try:
        from test_compliance_officer import TestComplianceOfficerAgent
        import pytest
        
        print("? Loading comprehensive test suite...")
        
        # Run the test suite
        print("🚀 Running Compliance Officer test suite...")
        result = pytest.main([
            f"{tests_path}/test_compliance_officer.py", 
            "-v", 
            "--tb=short"
        ])
        
        if result == 0:
            print("✅ All Compliance Officer tests passed!")
        else:
            print("❌ Some tests failed. Check the output above for details.")
            
    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure you've implemented ComplianceOfficerAgent in src/compliance_officer_agent.py")

# Quick preview of available tests
try:
    from test_compliance_officer import TestComplianceOfficerAgent
    import inspect
    
    # Get all test methods
    test_methods = [method for method in dir(TestComplianceOfficerAgent) 
                   if method.startswith('test_')]
    
    print("📝 Preview of Comprehensive Compliance Officer Tests:")
    for method_name in test_methods[:5]:  # Show first 5
        method = getattr(TestComplianceOfficerAgent, method_name)
        doc = method.__doc__ or method_name.replace('_', ' ').title()
        print(f"   • {doc}")
    if len(test_methods) > 5:
        print(f"   ... and {len(test_methods) - 5} more tests")
    print("\n💡 These tests validate regulatory compliance requirements!")
    print("💡 Includes word limits, citations, and required elements!")
except Exception as e:
    print(f"ℹ️ Test suite will be available after implementing foundation_sar.py: {e}")

# Call the function
run_comprehensive_compliance_officer_tests()

In [ ]:
# COMPLETE AGENT TESTING - Two-Tier Approach
# Students: Use this to test both agents together

def complete_agent_testing_workflow():
    """
    Complete testing workflow using two-tier approach:
    
    TIER 1: Simple Smoke Tests (You write these)
    - Basic functionality verification
    - Quick sanity checks
    - Development debugging
    
    TIER 2: Comprehensive Test Suites (Pre-built for you)
    - Complex edge cases
    - Regulatory compliance validation
    - Professional-grade testing
    """
    print("🔬 Complete Agent Testing Workflow")
    print("=" * 50)
    
    print("\n📋 TIER 1: Simple Smoke Tests (DO FIRST)")
    print("   1. Write simple_risk_analyst_smoke_test() - verify basic functionality")
    print("   2. Write simple_compliance_officer_smoke_test() - verify basic functionality")
    print("   3. Fix any basic issues before moving to Tier 2")
    
    print("\n🧪 TIER 2: Comprehensive Test Suites (DO AFTER TIER 1 PASSES)")
    print("   1. Run comprehensive risk analyst test suite (10 comprehensive tests)")
    print("   2. Run comprehensive compliance officer test suite (10 comprehensive tests)")
    print("   3. Get detailed pass/fail results with specific feedback")
    
    print("\n💡 WHY THIS APPROACH?")
    print("   ✅ Tier 1: Quick feedback while developing")
    print("   ✅ Tier 2: Professional validation without writing complex tests")
    print("   ✅ Saves time: You focus on implementation, not test creation")
    print("   ✅ Better coverage: Our test suites test edge cases you might miss")

# Quick test runner when both agents are ready
def run_both_agents_quick_test():
    """Quick test of both agents using pre-built test suites"""
    print("🚀 Quick Test of Both Agents")
    print("📋 TODO: Uncomment when both agents are implemented")
    
    # Uncomment when ready:
    try:
        import pytest
        
        print("🔍 Running quick tests for both agents...")
        
        # Run a subset of tests for quick validation
        risk_result = pytest.main([
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_agent_initialization",
            f"{tests_path}/test_risk_analyst.py::TestRiskAnalystAgent::test_analyze_case_success",
            "-v"
        ])
        
        compliance_result = pytest.main([
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_agent_initialization", 
            f"{tests_path}/test_compliance_officer.py::TestComplianceOfficerAgent::test_generate_compliance_narrative_success",
            "-v"
        ])
        
        if risk_result == 0 and compliance_result == 0:
            print("🎉 Both agents working! Ready for full test suite testing!")
        else:
            print("⚠️ Fix issues before running comprehensive tests")
            if risk_result != 0:
                print("   🔍 Risk Analyst needs fixes")
            if compliance_result != 0:
                print("   📝 Compliance Officer needs fixes")
                
    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure both agents are implemented")

complete_agent_testing_workflow()
run_both_agents_quick_test()

## 🔗 Phase 4 Preview: Agent Integration

Once both agents are working, you'll integrate them into a complete workflow.

In [ ]:
# TODO: Preview of integrated workflow
# This will be fully implemented in the next notebook

def preview_integrated_workflow():
    """Preview of how agents will work together"""
    
    workflow_steps = [
        "1. 📊 Load and validate case data",
        "2. 🔍 Risk Analyst performs Chain-of-Thought analysis",
        "3. 👤 Human review and approval gate",
        "4. ✅ Compliance Officer generates ReACT narrative (if approved)",
        "5. 📄 Generate complete SAR document",
        "6. 📊 Log audit trail and efficiency metrics"
    ]
    
    print("🔗 Integrated SAR Processing Workflow:")
    for step in workflow_steps:
        print(step)
    
    print("\n💡 Key Benefits:")
    print("• Two-stage processing reduces AI costs")
    print("• Human oversight ensures regulatory compliance")
    print("• Complete audit trails for examination")
    print("• Standardized analytical approaches")

preview_integrated_workflow()

## 📝 Development Checklist - Two-Tier Testing Approach

### ✅ Risk Analyst Agent (Phase 2)
- [ ] Implement Chain-of-Thought system prompt
- [ ] Create `analyze_case` method with error handling
- [ ] Add JSON parsing and validation
- [ ] **TIER 1**: Write simple smoke test (verify basic functionality)
- [ ] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [ ] Fix any issues identified by test suite

### ✅ Compliance Officer Agent (Phase 3)  
- [ ] Implement ReACT system prompt
- [ ] Create `generate_compliance_narrative` method
- [ ] Add narrative validation (word count, terminology)
- [ ] **TIER 1**: Write simple smoke test (verify basic functionality)
- [ ] **TIER 2**: Run comprehensive pre-built test suite (10 comprehensive tests)
- [ ] Fix any issues identified by test suite

### ✅ Testing Strategy Benefits
- [ ] **Time Savings**: Focus on implementation, not complex test creation
- [ ] **Better Coverage**: Pre-built test suites test edge cases you might miss
- [ ] **Quick Feedback**: Simple smoke tests for rapid development cycles
- [ ] **Professional Validation**: Comprehensive test suites ensure production readiness
- [ ] **Regulatory Compliance**: Built-in checks for SAR requirements

### 💡 **Testing Workflow**
1. **Start with Tier 1**: Write simple smoke tests to verify your agents don't crash
2. **Fix basic issues**: Iterate quickly with simple tests during development
3. **Move to Tier 2**: Run comprehensive test suites when basic functionality works
4. **Analyze results**: Use detailed feedback to improve agent performance
5. **Iterate**: Refine prompts and logic based on test results

## 🚀 Next Steps

1. **Complete Agent Implementation**: Finish both agent classes in the src/ directory
2. **Run Two-Tier Testing**: Start with smoke tests, then comprehensive test suites
3. **Workflow Integration**: Move to the next notebook for complete system integration
4. **Human-in-the-Loop**: Implement decision gates and review processes

## 📊 Available Test Suites Summary

**Risk Analyst Test Suite (10 tests):**
- Agent initialization and configuration
- Case analysis with valid JSON responses
- JSON parsing and error handling
- System prompt structure validation
- API call parameter verification
- Helper method functionality
- Edge case handling

**Compliance Officer Test Suite (10 tests):**
- Agent initialization and configuration
- Narrative generation with valid responses
- Word count validation (≤120 words)
- Regulatory citations inclusion
- JSON parsing and error handling
- ReACT prompt structure validation
- API call parameter verification

**Ready to build intelligent agents with professional-grade testing! 🕵️‍♀️**